In [1]:
import gensim.summarization.bm25 
import gensim
import os
import numpy as np
import pickle
import spacy
import torch
import transformers
import pickle
import numpy as np
from nltk import SnowballStemmer
import re

In [174]:
import bs4
import urllib.request
import re
import timeit
import os
import spacy

class WebsiteData():
    
    def __init__(self, website,depth, pathName):#,website,parser = 'lxml'):
        '''
        Constructor of the class. We let the user choose the parser type, but keep it default to the fastest parser.
        We are keeping the bsObject private because the end user should NOT get access to any HTML information.
        '''
        self.website = website
        self.pageDict = {}
        self.depth = depth
        self.pathName = pathName
        '''
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
        except:
            print("Website not accessible : " ,self.website)
            self.__bsObject = None
            return None
        '''
            
    def parseBaseWebsite(self, parser = 'lxml'):
        try:
            sauce = urllib.request.urlopen(self.website)
            self.__bsObject = bs4.BeautifulSoup(sauce,parser)
            return 1
        except Exception as e:
            print("Website not accessible. Exception :  " ,e)
            self.__bsObject = None
            return None
    
    def __getBaseURL(self):
        '''
        Return the base url of the website being parsed
        '''
        count = 0
        base1 = []
        base = ''
        if re.findall('http[s]?', self.website):
            for i in self.website:
                if i == '/':
                    count= count+1
                if count == 3:
                    break
                base1.append(i)
        
            base = base.join(base1)
        
        return base
    
    def displayHtml(self):
        '''
        Display website in HTML Structure 
        '''
        print(self.__bsObject.prettify())
    
    def retrieveHrefs(self):
        '''
        Retrieve links from the HREF Tags in the webpage 
        '''
        self.__linksList = []
        baseUrl = self.__getBaseURL()
        print("base url : ", baseUrl)
        
       
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        
        
        for link in bodyContent.find_all('a'):
            #print(link)
            suffixURL = link.get('href')
            #print(type(suffixURL))
            print(suffixURL)
            
            if (suffixURL is not None) and ('www.investopedia.com' in suffixURL) and (suffixURL.endswith('.asp'))  :
                self.__linksList.append(str(suffixURL))
                
         
        return self.__linksList
    
    def saveWebsiteData(self, levels):
        '''
        Save the data from the website in a txt file. The method creates a file with the topic of the link as the file name.
        The code will be extended to store the data from all the links in the linksList in the future.
        '''
        #nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        
        pathName = self.pathName
        if levels == 0 :
            pathName = pathName
        else:
            for counter in range(levels):
                pathName = pathName +  "/level" + str(counter+1 )
                
        if not os.path.exists(os.path.join(os.getcwd(),pathName)):
            os.mkdir(os.path.join(os.getcwd(),pathName ))
                

        #headerContent = self.__bsObject.find_all("span", attrs = {'class': 'mntl-sc-block-heading__text'})
 
            
            
        
        #print(headerContent)
        bodyContent = self.__bsObject.find("div", attrs = {'id': 'mntl-sc-page_1-0'})
        paragraphs = bodyContent.find_all('p')
        #headers = headerContent.find_all('')

        print("writing data")        
        file_name = os.path.join(os.path.join(os.getcwd(), pathName), (self.website).split("/")[-1] + ".txt")
        print(file_name)
        
        maxParaLen = 0
        sumParaLen = 0
        nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        totalParaLen = len(paragraphs)
        #print(paragraphs)
        count = 0 
        prevHeadingText = ""
        fileText = ""
        for paragraph in paragraphs:
            fileName = (self.website).split("/")[-1]
            fileName = fileName.split(".asp")[0]
            #print(fileName)
            #with open( os.path.join(os.path.join(os.getcwd(), pathName) , fileName + str(count)+ ".txt"),"w+", encoding='utf-8') as f:
               
            
            if paragraph.find_parent('div').find_previous('h3') is not None:
                heading = paragraph.find_parent('div').find_previous('h3')
                if heading.find('span') is not None:
                    heading_text = heading.find('span').text
                    
                else:
                    heading_text = heading.text
            else:
                heading = paragraph.find_parent('div').find_previous('h1')
                heading_text = heading.text
            
            
            heading_text = heading_text.replace('\n', "")
            heading_clean = re.sub('[^A-Za-z0-9]+', '', heading_text)
            
            
            
            if heading_text != prevHeadingText:
                sizeBuffer = 0
                counter = 0
                
                paraText = heading_text+ ". " + paragraph.getText()
                fileText = paraText
                
                prevHeadingText = heading_text
                file_clean_name = heading_clean
                
                #sizeBuffer = len(fileText.split())
                doc = nlp(fileText)
                fileDoc = [tokens for tokens in doc]
                sizeBuffer = len(fileDoc)
                print(" sizeBuffer1 : ", sizeBuffer )
                with open( os.path.join(os.path.join(os.getcwd(), pathName) , file_clean_name + ".txt"),"w+", encoding='utf-8') as f:
                    f.write(fileText)
                    
                
            else:
                doc = nlp(paragraph.getText())
                paraDoc = [tokens for tokens in doc]
                #print("this condition :" , len(paragraph.getText().split()) + sizeBuffer)
                #if (len(paragraph.getText().split()) + sizeBuffer) > 460:
                if (len(paraDoc) + sizeBuffer) > 460:
                    
                    print ("special case :", heading_clean + str(counter+1) )
                    counter = counter + 1
                    
                    heading_clean =  heading_clean + str(counter)
                    
                    file_clean_name = heading_clean
                    fileText = heading_text+ ". " + paragraph.getText()
                    #sizeBuffer = len(fileText.split())
                    doc = nlp(fileText)
                    fileDoc = [tokens for tokens in doc]
                    sizeBuffer = len(fileDoc)
                    print(" sizeBuffer2 : ", sizeBuffer )
                    with open( os.path.join(os.path.join(os.getcwd(), pathName) , file_clean_name + ".txt"),"w+", encoding='utf-8') as f1:
                        f1.write(fileText)
                        
                    
                else:
                    #fileText = fileText + " " + paragraph.getText()
                    fileText = heading_text+ ". " + paragraph.getText()
                    #sizeBuffer = sizeBuffer + len(fileText.split())
                    doc = nlp(fileText)
                    fileDoc = [tokens for tokens in doc]
                    sizeBuffer = sizeBuffer + len(fileDoc)
                    print(" sizeBuffer3 : ", sizeBuffer )
                    with open( os.path.join(os.path.join(os.getcwd(), pathName) , file_clean_name + ".txt"),"a", encoding='utf-8') as f2:
                        f2.write(fileText)
                        
                
           
            
            '''
            paraText = paragraph.getText()
            print("paraText   :   ", paraText)
            paraTextDoc = nlp(paraText)
            paraTextDocList = [tokens.text for tokens in paraTextDoc if tokens.text.strip() != '']
            paraLen = len(paraTextDocList)
            paraText = " ".join(paraTextDocList)
            sumParaLen = sumParaLen + paraLen
            if paraLen > maxParaLen:
                maxParaLen = paraLen

            f.write(paraText)
            '''
            
            count+=1
                

            #avgParaLen = (sumParaLen/totalParaLen)
            #self.pageDict[(self.website).split("/")[-1]] = (maxParaLen , avgParaLen)

            #f.close()
        return self.pageDict
        


class WebsiteParser:
    
    def __init__(self,baseWebsite,level, pathName):
        
        self.baseWebsite = baseWebsite
        self.level = level
        self.pathName = pathName
    
    def parseWebsite(self):
        
        pageDict ={}
        linkDict = {}
        
        for level in range(self.level):
            
            if level == 0:
                obj = WebsiteData(self.baseWebsite,level, self.pathName)
                objret = obj.parseBaseWebsite()
                linkDict[level +1] = obj.retrieveHrefs()
                if objret is not None: 
                    data = obj.saveWebsiteData(level) 
                    pageDict.update(data)
            else:
                linkDict[level +1] = []
        
                if level < (self.level - 1):
                
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                       
                        linkDict[level +1].extend(obj1.retrieveHrefs())
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)
                
                else:
                    for link in set(linkDict[level]):
                        obj1 = WebsiteData(link, level, self.pathName)
                        obj1ret = obj1.parseBaseWebsite()
                        
                        if obj1ret is not None: 
                            data = obj1.saveWebsiteData(level) 
                            pageDict.update(data)

In [175]:
numLevels = 3
initLink = 'https://www.investopedia.com/terms/r/riskmanagement.asp'
#initLink = "https://www.investopedia.com/terms/e/equity.asp"
path = r"C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus"
obj = WebsiteParser(initLink,numLevels,path)
print(obj.parseWebsite())

base url :  https://www.investopedia.com
https://www.investopedia.com/terms/f/financialplanner.asp
https://www.investopedia.com/terms/r/risk-analysis.asp
https://www.investopedia.com/ask/answers/042415/what-difference-between-moral-hazard-and-adverse-selection.asp
https://www.investopedia.com/articles/pf/07/risk_tolerance.asp
None
https://www.investopedia.com/terms/u/ustreasury.asp
https://www.investopedia.com/terms/d/derivative.asp
https://www.investopedia.com/terms/o/option.asp
https://www.investopedia.com/terms/f/futures.asp
https://www.investopedia.com/terms/s/subprime-meltdown.asp
https://www.investopedia.com/terms/m/mbs.asp
None
https://www.investopedia.com/terms/b/benchmark.asp
https://www.investopedia.com/terms/v/volatility.asp
None
https://www.investopedia.com/terms/s/standarddeviation.asp
https://www.investopedia.com/terms/d/dispersion.asp
https://www.investopedia.com/terms/a/annualized-total-return.asp
https://www.investopedia.com/terms/s/sp500.asp
None
https://www.investope

 sizeBuffer1 :  95
 sizeBuffer3 :  175
 sizeBuffer3 :  189
 sizeBuffer3 :  204
 sizeBuffer3 :  215
 sizeBuffer3 :  229
 sizeBuffer3 :  243
 sizeBuffer3 :  256
 sizeBuffer3 :  268
 sizeBuffer3 :  365
 sizeBuffer3 :  450
special case : AdvantagesandDisadvantagesofETFs1
 sizeBuffer2 :  82
 sizeBuffer3 :  202
 sizeBuffer3 :  256
 sizeBuffer3 :  365
 sizeBuffer1 :  36
 sizeBuffer3 :  129
 sizeBuffer3 :  227
 sizeBuffer3 :  380
 sizeBuffer3 :  445
special case : ETFCreationandRedemption1
 sizeBuffer2 :  41
 sizeBuffer3 :  103
 sizeBuffer3 :  216
base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/a/asset-backedsecurity.asp
None
https://www.investopedia.com/terms/g/gse.asp
https://www.investopedia.com/terms/c/credit-agency.asp
None
https://www.investopedia.com/terms/t/tranches.asp
None
https://www.investopedia.com/terms/s/subprime-meltdown.asp
https://www.investopedia.com/terms/l/lehman-brothers.asp
https://www.investopedia.com/terms/s/subprime_mortgage.asp
https:

 sizeBuffer3 :  172
 sizeBuffer1 :  13
 sizeBuffer1 :  27
 sizeBuffer3 :  179
 sizeBuffer3 :  235
 sizeBuffer3 :  349
 sizeBuffer3 :  445
special case : BiasesStudiedinBehavioralFinance1
 sizeBuffer2 :  67
 sizeBuffer1 :  76
 sizeBuffer3 :  145
 sizeBuffer3 :  245
base url :  https://www.investopedia.com
None
https://www.investopedia.com/terms/f/financialinstrument.asp
https://www.investopedia.com/terms/d/derivative.asp
https://www.investopedia.com/ask/answers/difference-between-options-and-futures/
https://www.investopedia.com/terms/c/call.asp
https://www.investopedia.com/terms/p/put.asp
https://www.investopedia.com/best-brokers-for-options-trading-4587876
None
https://www.investopedia.com/terms/o/option-premium.asp
https://www.investopedia.com/terms/o/optionscontract.asp
https://www.investopedia.com/terms/s/strikeprice.asp
https://www.investopedia.com/terms/e/expirationdate.asp
https://www.investopedia.com/ask/answers/sell-open-buy-close-buy-open-sell-close-mean/
https://www.investop

 sizeBuffer1 :  76
 sizeBuffer3 :  127
 sizeBuffer3 :  229
 sizeBuffer3 :  301
 sizeBuffer3 :  355
 sizeBuffer1 :  92
 sizeBuffer3 :  142
 sizeBuffer1 :  11
 sizeBuffer3 :  81
 sizeBuffer1 :  22
 sizeBuffer3 :  113
 sizeBuffer1 :  102
 sizeBuffer3 :  142
 sizeBuffer3 :  200
 sizeBuffer1 :  25
 sizeBuffer3 :  81
 sizeBuffer1 :  94
 sizeBuffer3 :  201
 sizeBuffer3 :  226
 sizeBuffer1 :  53
 sizeBuffer3 :  179
 sizeBuffer3 :  239
 sizeBuffer3 :  342
 sizeBuffer3 :  378
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\liability.asp.txt
 sizeBuffer1 :  114
 sizeBuffer3 :  214
 sizeBuffer3 :  315
 sizeBuffer1 :  87
 sizeBuffer3 :  124
 sizeBuffer3 :  187
 sizeBuffer3 :  225
 sizeBuffer1 :  105
 sizeBuffer3 :  224
 sizeBuffer1 :  54
 sizeBuffer3 :  91
 sizeBuffer3 :  112
 sizeBuffer3 :  135
 sizeBuffer3 :  154
 sizeBuffer1 :  72
 sizeBuffer3 :  118
 sizeBuffer3 :  185
 sizeBuffer3 :  228
 sizeBuffer1 :  96
 sizeBuffer3 :  183
 sizeBuffer3 :  263
 sizeBuf

C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\incometax.asp.txt
 sizeBuffer1 :  90
 sizeBuffer1 :  85
 sizeBuffer3 :  157
 sizeBuffer3 :  191
 sizeBuffer1 :  172
 sizeBuffer1 :  56
 sizeBuffer1 :  68
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\mentalaccounting.asp.txt
 sizeBuffer1 :  72
 sizeBuffer1 :  97
 sizeBuffer3 :  182
 sizeBuffer3 :  335
 sizeBuffer1 :  26
 sizeBuffer1 :  112
 sizeBuffer3 :  210
 sizeBuffer3 :  322
 sizeBuffer1 :  46
 sizeBuffer1 :  104
 sizeBuffer3 :  251
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\holdingperiod.asp.txt
 sizeBuffer1 :  95
 sizeBuffer1 :  75
 sizeBuffer3 :  156
 sizeBuffer1 :  89
 sizeBuffer3 :  162
 sizeBuffer3 :  180
 sizeBuffer3 :  276
 sizeBuffer1 :  122
 sizeBuffer1 :  44
 sizeBuffer3 :  152
 sizeBuffer3 :  231
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\europeanoption.asp.txt
 size

...

C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\returnonequity.asp.txt
 sizeBuffer1 :  59
 sizeBuffer3 :  88
 sizeBuffer1 :  51
 sizeBuffer3 :  98
 sizeBuffer3 :  162
 sizeBuffer3 :  224
 sizeBuffer3 :  265
 sizeBuffer1 :  95
 sizeBuffer3 :  183
 sizeBuffer3 :  264
 sizeBuffer1 :  89
 sizeBuffer3 :  139
 sizeBuffer1 :  88
 sizeBuffer3 :  137
 sizeBuffer3 :  232
 sizeBuffer3 :  296
 sizeBuffer1 :  61
 sizeBuffer3 :  134
 sizeBuffer1 :  54
 sizeBuffer3 :  112
 sizeBuffer3 :  217
 sizeBuffer3 :  308
 sizeBuffer3 :  356
 sizeBuffer3 :  438
special case : UsingROEtoIdentifyProblems1
 sizeBuffer2 :  79
 sizeBuffer1 :  44
 sizeBuffer3 :  114
 sizeBuffer1 :  81
 sizeBuffer1 :  39
 sizeBuffer3 :  51
 sizeBuffer3 :  140
 sizeBuffer3 :  160
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\negative-shareholder-equity.asp.txt
 sizeBuffer1 :  75
 sizeBuffer3 :  93
 sizeBuffer3 :  110
 sizeBuffer3 :  148
 sizeBuffer1 :  70
 sizeBuffer3

C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\3-s-simple-investing.asp.txt
 sizeBuffer1 :  64
 sizeBuffer1 :  28
 sizeBuffer3 :  150
 sizeBuffer3 :  228
 sizeBuffer3 :  275
 sizeBuffer3 :  371
 sizeBuffer3 :  462
special case : UnderstandingtheInvestmentRiskLadder1
 sizeBuffer2 :  86
 sizeBuffer3 :  121
 sizeBuffer3 :  200
 sizeBuffer3 :  280
 sizeBuffer3 :  303
 sizeBuffer1 :  84
 sizeBuffer3 :  176
 sizeBuffer1 :  76
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\binomialoptionpricing.asp.txt
 sizeBuffer1 :  61
 sizeBuffer1 :  30
 sizeBuffer1 :  80
 sizeBuffer3 :  138
 sizeBuffer3 :  293
 sizeBuffer1 :  56
 sizeBuffer3 :  160
 sizeBuffer3 :  214
 sizeBuffer3 :  319
 sizeBuffer1 :  62
 sizeBuffer3 :  139
 sizeBuffer3 :  202
 sizeBuffer3 :  277
 sizeBuffer3 :  322
 sizeBuffer3 :  413
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\least-squares-method.asp.txt
 sizeBuffer1 :  66

C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\how-can-derivatives-be-used-risk-management.asp.txt
 sizeBuffer1 :  92
 sizeBuffer1 :  63
 sizeBuffer3 :  130
 sizeBuffer1 :  70
 sizeBuffer1 :  95
 sizeBuffer3 :  169
 sizeBuffer1 :  70
 sizeBuffer3 :  143
 sizeBuffer1 :  110
 sizeBuffer1 :  62
 sizeBuffer3 :  182
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\strangle-strategy.asp.txt
 sizeBuffer1 :  56
 sizeBuffer3 :  97
 sizeBuffer3 :  215
 sizeBuffer1 :  48
 sizeBuffer3 :  89
 sizeBuffer1 :  20
 sizeBuffer3 :  105
 sizeBuffer3 :  185
 sizeBuffer3 :  301
 sizeBuffer3 :  388
special case : FactorsThatInfluenceAllStrangles1
 sizeBuffer2 :  91
 sizeBuffer1 :  87
 sizeBuffer3 :  95
 sizeBuffer3 :  156
 sizeBuffer3 :  223
 sizeBuffer3 :  231
 sizeBuffer1 :  72
 sizeBuffer3 :  80
 sizeBuffer3 :  165
 sizeBuffer3 :  215
 sizeBuffer1 :  103
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level

 sizeBuffer3 :  137
 sizeBuffer1 :  106
 sizeBuffer3 :  188
 sizeBuffer1 :  70
 sizeBuffer1 :  78
 sizeBuffer3 :  149
 sizeBuffer1 :  64
 sizeBuffer3 :  155
 sizeBuffer1 :  80
 sizeBuffer1 :  72
 sizeBuffer3 :  119
 sizeBuffer1 :  68
 sizeBuffer3 :  135
 sizeBuffer1 :  60
 sizeBuffer3 :  96
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\how-can-you-calculate-volatility-excel.asp.txt
 sizeBuffer1 :  69
 sizeBuffer3 :  159
 sizeBuffer3 :  253
 sizeBuffer3 :  302
 sizeBuffer3 :  417
 sizeBuffer3 :  440
special case : HowDoYouCalculateVolatilityinExcel1
 sizeBuffer2 :  56
 sizeBuffer1 :  132
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\anchoring.asp.txt
 sizeBuffer1 :  38
 sizeBuffer1 :  55
 sizeBuffer3 :  165
 sizeBuffer1 :  79
 sizeBuffer3 :  142
 sizeBuffer3 :  192
 sizeBuffer3 :  273
 sizeBuffer3 :  305
 sizeBuffer1 :  66
 sizeBuffer3 :  86
 sizeBuffer3 :  160
 sizeBuffer3 :  240
writing data
C:\Users\p

 sizeBuffer3 :  151
 sizeBuffer3 :  204
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\fdic.asp.txt
 sizeBuffer1 :  103
 sizeBuffer3 :  248
 sizeBuffer1 :  55
 sizeBuffer3 :  122
 sizeBuffer1 :  30
 sizeBuffer3 :  89
 sizeBuffer3 :  106
 sizeBuffer1 :  39
 sizeBuffer1 :  83
 sizeBuffer3 :  148
 sizeBuffer3 :  182
 sizeBuffer1 :  66
 sizeBuffer3 :  113
 sizeBuffer1 :  72
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\bluechip.asp.txt
 sizeBuffer1 :  72
 sizeBuffer1 :  25
 sizeBuffer1 :  104
 sizeBuffer3 :  213
 sizeBuffer3 :  302
 sizeBuffer3 :  412
 sizeBuffer1 :  185
 sizeBuffer3 :  280
 sizeBuffer1 :  88
 sizeBuffer3 :  201
writing data
C:\Users\prateek.sethi\Desktop\BERT\Website_QA\invest_corpus/level1/level2\hedgeratio.asp.txt
 sizeBuffer1 :  60
 sizeBuffer3 :  93
 sizeBuffer1 :  15
 sizeBuffer1 :  58
 sizeBuffer3 :  108
 sizeBuffer1 :  60
 sizeBuffer3 :  152
 sizeBuffer1 :  84
 sizeBuffer3 :  197
wri

In [2]:
class DocumentRanker():
    
    def __init__(self, query, folderPath):
        
        self.documentFolderPath = folderPath
        print("folderPath  : ", folderPath)
        self.query = self.convertQuery(query)
        self.folderDocumentContent = self.loadDocuments()  # Store the content of all the files in dictionary
        self.bm25Model = self.createBM25Model()
    
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
        
    def loadDocuments(self):
        '''
        Load documents from the invest_data folder into a dictionary
        '''
        documentTuple = []
        #nounList = self.getNouns()
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        fileList = []
        for root, dirs, files in os.walk(self.documentFolderPath):
    
            for file in files:
                if file.endswith('txt'):
                    with open(os.path.abspath(os.path.join(root, file)), 'r' , encoding="utf8" ) as f:
                        content = f.read()
                        content = content.lower()
                        doc = nlp(content)
                        tokenizedContent = [tokens.text for tokens in doc if tokens.text]         # Change 1 
                        
                        if file not in fileList:
                            fileList.append(file)
                            documentTuple.append((file,tokenizedContent))
        
       
        return documentTuple
    
        
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                print(token.text)
                print(token.pos_)
                nounList.append(token.text)
            
        print("noun list : ", nounList)        
        return nounList
    
        
    def createBM25Model(self):
        '''
        Return a created BM25 model.
        '''
        return gensim.summarization.bm25.BM25([x[1] for x in self.folderDocumentContent])
    
    def rankDocuments(self):
        '''
        Rank the documents in the corpus wrt the query. 
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        scores = self.bm25Model.get_scores(query)
        documentScores =  [(self.folderDocumentContent[i][0],scores[i]) for i in range(len(scores))]
        return documentScores
    
    def returnTopK(self,k=5):
        '''
        Return top k ranked documents wrt given user query
        '''
        
        scoreDict = self.rankDocuments()
        return sorted(scoreDict , key = lambda x: x[1], reverse = True)[:k]
    
    def returnTopDocuments(self):
        '''
        Return top documents wrt given user query. This uses a different approach from our 
        previous function. We return all documents with score a standard deviation above the
        mean score of our corpus
        '''
        
        scoreTup = self.rankDocuments()
        #scoreDict = dict(scoreTup)
        scores = np.array([x[1] for x in scoreTup])
        #scores = np.array(list(scoreDict.values()))
        meanScore = np.mean(scores)
        sdScore = np.std(scores)
        maxThreshold = meanScore + sdScore
        topScoresDict = [x for x in scoreTup if x[1] > maxThreshold]
        topDocumentScores = sorted(topScoresDict, key = lambda x: x[1], reverse = True)
        
        return topDocumentScores
    
    def returnTopDocumentsData(self):
        '''
        Return data from the top documents retrieved by returnTopDocuments
        '''
        #query = self.query
        #topDocumentScores = self.returnTopDocuments()
        topDocumentScores = self.returnTopK(5)
        print("topDocumentScores  : ", topDocumentScores)
        topDocumentNames = [x[0] for x in topDocumentScores]
        #print(topDocumentNames)
        
        topDocumentText = [x for x in self.folderDocumentContent if x[0] in topDocumentNames]
        return topDocumentText

In [3]:
class PassageRanker():
    
    def __init__(self, query, topDocumentContent):
        
        self.query = self.convertQuery(query)
        self.topDocumentContent = topDocumentContent
        #self.BM25Model = self.createBM25Model()
    
    def convertQuery(self,query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc
    
    
    def createBM25Model(self,documentContent):
        '''
        Return a created BM25 model.
        '''
        #return gensim.summarization.bm25.BM25([x[1] for x in self.topDocumentContent])
        return gensim.summarization.bm25.BM25(documentContent)
    
    
    def getNouns(self):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in self.query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
    def checkNounMatch(self,paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        paragraphContent = " ".join([stemmer.stem(token) for token in paragraph])
        
        
        for nouns in nounList:
            
            #paragraphDoc = nlp(paragraphContent)
            #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc]
            
            if stemmer.stem(nouns) in paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
    def returnParagraphList(self,content):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        
        paragraphList = []        
        paragraphList.append(content)
                        
        return paragraphList
        
        '''
        paragraphList = []
        paragraph = content #[sepCounter:i] # " ".join() <-
        nounList = self.getNouns()
        if self.checkNounMatch(paragraph,nounList):     
            paragraphList.append(paragraph)
        return paragraphList
        '''
        
        
        
   
    
    
    def returnParagraphScores(self,bm25Model,query):
        '''
        Return the score of a paragraph given a model.
        '''
        return bm25Model.get_scores(query)
        
    
    def returnTopPassages(self, k=10):
        '''
        Rank each paragraph from the document and return the top 10 passages from the collection of documents
        '''
        query = [tokens.text for tokens in self.query if not tokens.is_punct]
        
        #documentParagraphScores = [(document[0],self.rankPassages(document[1])) for document in topDocumentContent]
        paragraphScoreTup = []
        paragraphLoL = []
        for documents in self.topDocumentContent:
            #print(documents)
            paragraphLoL.extend(self.returnParagraphList(documents[1]))
        
        
        print(len(paragraphLoL))
        #passageBM25Model = self.createBM25Model(paragraphLoL)
        
        #for documents in self.topDocumentContent:
        #paragraphScores = self.returnParagraphScores(passageBM25Model, query)
        for i,paragraph in enumerate(paragraphLoL):
                
            #paragraphScoreTup.append((paragraph,paragraphScores[i]))
            paragraphScoreTup.append((paragraph,1))
        
        print(" length of paragraphScoreTup : ", len(paragraphScoreTup))
        
        self.topParagraphScores = sorted(paragraphScoreTup, key = lambda x: x[1], reverse = True)
        
        return self.topParagraphScores#[:k]

In [42]:
class bertQAModel():
    
    def __init__(self):
        
        self.tokenizerModel,self.bertQAModel = self.__initializeModel()
    
    def __initializeModel(self):
        '''
        Initialize the Beret Tokenizer and the QA model. Note that this is currently compatible with
        transformers module, NOT pytorch/tensorflow
        '''
        with open("./models/bertTokenizer.pkl","rb") as f:
            tokenizerModel = pickle.load(f)
        
        with open("./models/bertQAModel.pkl","rb") as f:
            bertQAModel = pickle.load(f)
        
        return tokenizerModel, bertQAModel
    
    def stringProcess(self,answer):
        answerSplit = answer.split(" ##")
        return "".join(answerSplit)
    
    def predict(self,text,question):
        '''
        Predict the answer given a passage and a question.
        '''
        input_text = "[CLS] " + question + " [SEP] " + text + " [SEP]"
        print("INPUT_TEXT : ")
        print(input_text)
        input_ids = self.tokenizerModel.encode(input_text)
        #print("TOKENIZED TEXT : ")
        #print(input_ids)
        token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]
        #print("TOKEN TYPE IDS : ")
        #print(token_type_ids)
        if len(input_ids) < 512:
            start_scores, end_scores = self.bertQAModel(torch.tensor([input_ids]), token_type_ids=torch.tensor([token_type_ids]))
        
        
            # Normalized scores
            #startScoresVec = ( (startScoresVec - np.mean(startScoresVec))/np.std(startScoresVec))
            #endScoresVec = ( (endScoresVec - np.mean(endScoresVec))/np.std(endScoresVec))

            # Normalized Scores
            #startScoresVec = scale(np.reshape(startScoresVec,newshape=(-1,1)))
            #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))

            # Normalized Scores
            #softmax = torch.nn.Softmax()
            #start_scores = softmax(start_scores)
            #end_scores = softmax(end_scores)
            #endScoresVec = scale(np.reshape(endScoresVec,newshape=(-1,1)))

            #startScoresVec = ( (startScoresVec - np.min(startScoresVec))/np.ptp(startScoresVec))
            #endScoresVec = ( (endScoresVec - np.min(endScoresVec))/np.ptp(endScoresVec))


            all_tokens = self.tokenizerModel.convert_ids_to_tokens(input_ids)
            startScoresVec = start_scores.detach().numpy().flatten()
            endScoresVec = end_scores.detach().numpy().flatten()
            
            
            startScore,endScore = np.sort(startScoresVec)[-2],np.sort(endScoresVec)[-2]
            
            answer = ' '.join(all_tokens[int(np.where(startScoresVec == startScore)[0][0]) : int(np.where(endScoresVec == endScore)[0][0])+1])
            
            #startScore = 
            
            
            '''
            startScoreInd,endScoreInd = torch.argmax(start_scores),torch.argmax(end_scores)
            startScoresVec = start_scores.detach().numpy().flatten()
            endScoresVec = end_scores.detach().numpy().flatten()
            '''
            #startScoreMax, endScoreMax = startScoresVec[startScoreInd] , endScoresVec[endScoreInd]
            #avgScore = float(np.absolute(startScoreMax) + np.absolute(endScoreMax))
            #avgScore = float(startScoreMax + endScoreMax)
            print("ANSWER RETRIEVED : ")
            print(self.stringProcess(answer))
            return (self.stringProcess(answer),endScore - startScore)
        else:
            return ("", 0.0)

In [28]:
# C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data
# C:\Users\piyush.kathuria\Desktop\invest-data
path = r"C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data"

In [29]:
query = "How do I calculate volatility?"

In [30]:
docRanker = DocumentRanker(query,path)
topDocuments = docRanker.returnTopDocumentsData()

folderPath  :  C:\Users\piyush.kathuria\Documents\Neural_IR\final_app\invest-data
topDocumentScores  :  [('HowCanICalculatetheDeltaAdjustedNotionalValue.txt', 14.78778496375849), ('HowDoYouCalculateVolatilityinExcel1.txt', 13.71024994566791), ('HowtoCalculateVolatility.txt', 10.953973349704798), ('HowDoYouCalculateVolatilityinExcel.txt', 10.778136850778946), ('HowDoYouCalculateValueatRiskVaRinExcel.txt', 9.206701654837794)]


In [31]:
passageRanker = PassageRanker(query , topDocuments)
passageRanking = passageRanker.returnTopPassages()

5
 length of paragraphScoreTup :  5


In [32]:
passageRanking

[([' ',
   'how',
   'to',
   'calculate',
   'volatility',
   '.',
   'volatility',
   'is',
   'often',
   'calculated',
   'using',
   'variance',
   'and',
   'standard',
   'deviation',
   '.',
   'the',
   'standard',
   'deviation',
   'is',
   'the',
   'square',
   'root',
   'of',
   'the',
   'variance',
   '.',
   '\xa0 ',
   'for',
   'simplicity',
   ',',
   'let',
   "'s",
   'assume',
   'we',
   'have',
   'monthly',
   'stock',
   'closing',
   'prices',
   'of',
   '$',
   '1',
   'through',
   '$',
   '10',
   '.',
   'for',
   'example',
   ',',
   'month',
   'one',
   'is',
   '$',
   '1',
   ',',
   'month',
   'two',
   'is',
   '$',
   '2',
   ',',
   'and',
   'so',
   'on',
   '.',
   '\xa0',
   'to',
   'calculate',
   'variance',
   ',',
   'follow',
   'the',
   'five',
   'steps',
   'below',
   '.',
   'in',
   'this',
   'case',
   ',',
   'the',
   'resulting',
   'variance',
   'is',
   '$',
   '8.25',
   '.',
   'the',
   '\xa0',
   'square',
   'ro

In [43]:
bertQA = bertQAModel()
answerTuple = []
nlp = spacy.load("en_core_web_sm")
queryDoc = nlp(query)
query = " ".join([tokens.text for tokens in queryDoc if not tokens.is_punct])

for passage,_ in passageRanking : 
            
    paraText = " ".join(passage)
    answerTuple.append(bertQA.predict(paraText, query))

INPUT_TEXT : 
[CLS] How do I calculate volatility [SEP]   how to calculate volatility . volatility is often calculated using variance and standard deviation . the standard deviation is the square root of the variance .    for simplicity , let 's assume we have monthly stock closing prices of $ 1 through $ 10 . for example , month one is $ 1 , month two is $ 2 , and so on .   to calculate variance , follow the five steps below . in this case , the resulting variance is $ 8.25 . the   square root is taken to get the standard deviation . this equals $ 2.87 . this is a measure of risk , and shows how values are spread out around the average price . it gives traders an idea of how far the price may deviate from the average .    if prices are randomly distributed ( and often they are not ) , then about 68 % off all data values will fall within one standard deviation . 95 % of data values will fall within two standard deviations ( 2 x 2.87   in our example ) , and 99.7 % of all values will fa

In [44]:
print("ANSWERS : ")
sortedAnswers = sorted(answerTuple, key = lambda x: x[1], reverse = True)

for item in sortedAnswers:
    print(item)

ANSWERS : 
('excel ?', 0.80091995)
('excel ?', 0.79056966)
('determine the time frame for which the metric will be computed .', -0.19503736)
('how to calculate volatility . volatility is often calculated using variance and standard deviation . the standard deviation is the square root of the variance', -0.42735553)
('', -1.3246583)


In [552]:
## Timing Stemmer and Lemmatizer in both spacy and nltk ##

document = "Investors use a variety of tactics to ascertain risk. One of the most commonly used absolute risk metrics is standard deviation, a statistical measure of dispersion around a central tendency. You look at the average return of an investment and then find its average standard deviation over the same time period. Normal distributions (the familiar bell-shaped curve) dictate that the expected return of the investment is likely to be one standard deviation from the average 67% of the time and two standard deviations from the average deviation 95% of the time. This helps investors evaluate risk numerically. If they believe that they can tolerate the risk, financially and emotionally, they invest."

In [119]:
document = " Time      .    Hey"

In [203]:
## 1. Spacy Lemmatizer ##
nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])

In [204]:
from time import time

start = time()
documentDoc = nlp(document)
nounLemma = [tokens.lemma_ for tokens in documentDoc if tokens.text.strip() != '']
end = time()
print(nounLemma)
print("TIME : ",end - start)

['Time', '.', 'hey']
TIME :  0.01604437828063965


In [205]:
len(topDocuments)

33

In [271]:
def convertQuery(query):
        
        nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
        doc = nlp(query.lower())
        return doc

def getNouns(query):
        
        #nlp = spacy.load("en_core_web_sm")
        
        '''
        spacy_stopwords = list(spacy.lang.en.stop_words.STOP_WORDS)
        
        querySplit = self.query.lower().split()
        
        querySplit = [x for x in querySplit if x not in spacy_stopwords]
        
        print(querySplit)
        
        return querySplit
        
        '''
        nounList = []
        for token in query:
            #print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop)
            if token.pos_ == "NOUN" or token.pos_ == "PROPN" or token.pos_ == "ADJ":
                nounList.append(token.text)
            
        #print("noun list : ", nounList)        
        return nounList
    
def checkNounMatch(paragraph,nounList):
        '''
        A function to see if each noun is matching for a paragraph
        '''
        nounBool = True
        
        #nlp = spacy.load("en_core_web_sm",disable=['ner', 'parser', 'textcat'])
        stemmer = SnowballStemmer("english")
        #paragraphContent = " ".join(paragraph)
        stemmedPara = [stemmer.stem(token) for token in paragraph]
        #paragraphDoc = nlp(paragraphContent)
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc] 
        #nounDoc = nlp(" ".join(nounList))
        #nounsLemma = [tokens.lemma_ for tokens in nounDoc]
        #paragraphLemma = [tokens.lemma_ for tokens in paragraphDoc if tokens.pos_ in ["NOUN","ADJ"]]
        for nouns in nounList: #nounsLemma:
            
            #nounLemma = [tokens.lemma_ for tokens in nounDoc]
            print(stemmer.stem(nouns))
            if stemmer.stem(nouns) in stemmedPara:#paragraphContent:
                nounBool = True
            else:
                nounBool = False
                break
        
        return nounBool
                
            
def returnParagraphList(content,nounList):
        '''
        Return pargaraph in the form of list of lists to be fed to the BM25 Model
        '''
        paragraphList = []
        sepCounter = 0
        #nounList = getNouns()
        
        for i,term in enumerate(content):
            
            if term == "--------------------------":
                paragraph = content[sepCounter:i] # " ".join() <-
                if checkNounMatch(paragraph,nounList):
                #if all(nouns in paragraph for nouns in nounList):
                    paragraphList.append(paragraph)
                
                sepCounter = i+1
        
        return paragraphList

In [272]:
len(topDocuments)

54

In [273]:
query = "What is implied volatility?"

In [262]:
queryList = convertQuery(query)

In [264]:
queryList

what are two common types of mbs?

In [265]:
nounList = getNouns(queryList)

In [266]:
nounList

['common', 'types', 'mbs']

In [267]:
for documents in topDocuments :
    
    
    start = time()
    paragraphList = returnParagraphList(documents[1],nounList)
    end = time()
    print("DOCUMENT : ",documents[0], " , TIME TAKEN : ",end - start)

DOCUMENT :  riskmanagement.asp.txt  , TIME TAKEN :  0.04189729690551758
DOCUMENT :  basispoint.asp.txt  , TIME TAKEN :  0.017946243286132812
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.061411142349243164
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.051119089126586914
DOCUMENT :  mbs.asp.txt  , TIME TAKEN :  0.025313138961791992
DOCUMENT :  prospecttheory.asp.txt  , TIME TAKEN :  0.017072677612304688
DOCUMENT :  balancesheet.asp.txt  , TIME TAKEN :  0.03179740905761719
DOCUMENT :  bell-curve.asp.txt  , TIME TAKEN :  0.01796889305114746
DOCUMENT :  benchmarkerror.asp.txt  , TIME TAKEN :  0.009974241256713867
DOCUMENT :  calculating-covariance.asp.txt  , TIME TAKEN :  0.02587723731994629
DOCUMENT :  certificateofdeposit.asp.txt  , TIME TAKEN :  0.1207284927368164
DOCUMENT :  correlationcoefficient.asp.txt  , TIME TAKEN :  0.021047353744506836
DOCUMENT :  derivative.asp.txt  , TIME TAKEN :  0.06382894515991211
DOCUMENT :  equity.asp.txt  , TIME TAKEN :  0.05587577819824219
DOCUMENT 

In [236]:
stemmer = SnowballStemmer("english")

In [268]:
[stemmer.stem(x) for x in nounList]

['common', 'type', 'mbs']

In [275]:
document = "Types of Mortgage - Backed Securities . There are two common types of MBSs : pass - throughs and collateralized mortgage obligations ( CMO ) ."

In [276]:
document.split()

['Types',
 'of',
 'Mortgage',
 '-',
 'Backed',
 'Securities',
 '.',
 'There',
 'are',
 'two',
 'common',
 'types',
 'of',
 'MBSs',
 ':',
 'pass',
 '-',
 'throughs',
 'and',
 'collateralized',
 'mortgage',
 'obligations',
 '(',
 'CMO',
 ')',
 '.']

In [274]:
checkNounMatch(["There","are","two","common","types","of","mbss"],nounList)

common
type
mbs


False

In [367]:
stem = SnowballStemmer('english')

In [579]:
stem.stem('security')

'secur'

In [514]:
nlp = spacy.load("en_core_web_sm")

In [121]:
doc = nlp("What is margin call definition")

In [122]:
[tokens.pos_ for tokens in doc]

['PRON', 'VERB', 'NOUN', 'NOUN', 'NOUN']

In [585]:
x = "Securities there!!!"

In [586]:
doc = nlp(x)

In [587]:
query = [tokens.lemma_ for tokens in doc]

In [588]:
query

['security', 'there', '!', '!', '!']

TypeError: 'bool' object is not iterable

In [41]:
x = [1,2,3,4,4]
print([a for a in x while(a<3)]

SyntaxError: invalid syntax (<ipython-input-41-b2f087915276>, line 2)

In [7]:
"   dsf    fdf   ".replace(" ","")

'dsffdf'

In [45]:
os.getcwd()

'C:\\Users\\prateek.sethi\\Desktop\\BERT\\Website_QA\\updated_final_app'

In [52]:
t1 = "   the black scholes formula . the mathematics involved in the formula are complicated and can be intimidating . fortunately , you do n't need to know or even understand the math to use   black - scholes modeling in your own strategies . options traders have access to a variety of online options calculators , and many of today 's trading platforms boast robust options analysis tools , including indicators and spreadsheets that perform the calculations and output the options pricing values . the black scholes call option formula is calculated by multiplying the stock price by the cumulative standard normal probability distribution function . thereafter , the net present value ( npv ) of the strike price multiplied by the cumulative standard normal distribution is subtracted from the resulting value of the previous calculation . in mathematical notation : ﻿c = stn(d1)−ke−rtn(d2)where : d1=lnstk+(r+σv22 )   tσs   tandd2=d1−σs   twhere : c = call   option   prices = current   stock   ( or   other   underlying )   pricek = strike   pricer = risk - free   interest   ratet = time   to   maturityn = a   normal   distribution\begin{aligned } & c = s_t n(d _ 1 ) - k e ^{-rt } n(d _ 2)\\ & \textbf{where:}\\ & d_1 = \frac{ln\frac{s_t}{k } + ( r+ \frac{\sigma ^{2 } _ v}{2 } ) \ t}{\sigma_s \ \sqrt{t}}\\ & \text{and}\\ & d_2 = d _ 1 - \sigma_s \ \sqrt{t}\\ & \textbf{where:}\\ & c = \text{call option price}\\ & s = \text{current stock ( or other underlying ) price}\\ & k = \text{strike price}\\ & r = \text{risk - free interest rate}\\ & t = \text{time to maturity}\\ & n = \text{a normal distribution}\\ \end{aligned}​c = st​n(d1​)−ke−rtn(d2​)where : d1​=σs​   t​lnkst​​+(r+2σv2​​ )   t​andd2​=d1​−σs​   t​where : c = call   option   prices = current   stock   ( or   other   underlying )   pricek = strike   pricer = risk - free   interest   ratet = time   to   maturityn = a   normal   distribution​﻿ "

In [49]:
len(t1.split())

324

In [50]:
nlp = spacy.load("en_core_web_sm",disable = ['ner', 'parser', 'textcat'])
doc = nlp(t1)

In [51]:
t1doc= [tokens for tokens in doc]
len(t1doc)

378

In [9]:
331-283

48

In [14]:
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
nlp = English()
tokenizer = nlp.Defaults.create_tokenizer(nlp)

In [15]:
tokens = tokenizer(t1)

In [18]:
type(tokens)

spacy.tokens.doc.Doc

In [54]:
question = "what is black scholes model"

In [56]:
with open("../models/bertTokenizer.pkl","rb") as f:
    tokenizerModel = pickle.load(f)

In [59]:
input_text = "[CLS] " + question + " [SEP] " + t1 + " [SEP]"
print("INPUT_TEXT : ")
print(input_text)
input_ids = tokenizerModel.encode(input_text)
print("TOKENIZED TEXT : ")
print(input_ids)
print(len(input_ids))
token_type_ids = [0 if i <= input_ids.index(102) else 1 for i in range(len(input_ids))]

INPUT_TEXT : 
[CLS] what is black scholes model [SEP]    the black scholes formula . the mathematics involved in the formula are complicated and can be intimidating . fortunately , you do n't need to know or even understand the math to use   black - scholes modeling in your own strategies . options traders have access to a variety of online options calculators , and many of today 's trading platforms boast robust options analysis tools , including indicators and spreadsheets that perform the calculations and output the options pricing values . the black scholes call option formula is calculated by multiplying the stock price by the cumulative standard normal probability distribution function . thereafter , the net present value ( npv ) of the strike price multiplied by the cumulative standard normal distribution is subtracted from the resulting value of the previous calculation . in mathematical notation : ﻿c = stn(d1)−ke−rtn(d2)where : d1=lnstk+(r+σv22 )   tσs   tandd2=d1−σs   twhere 

Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors


TOKENIZED TEXT : 
[101, 2054, 2003, 2304, 8040, 19990, 2944, 102, 1996, 2304, 8040, 19990, 5675, 1012, 1996, 5597, 2920, 1999, 1996, 5675, 2024, 8552, 1998, 2064, 2022, 24439, 1012, 14599, 1010, 2017, 2079, 1050, 1005, 1056, 2342, 2000, 2113, 2030, 2130, 3305, 1996, 8785, 2000, 2224, 2304, 1011, 8040, 19990, 11643, 1999, 2115, 2219, 9942, 1012, 7047, 13066, 2031, 3229, 2000, 1037, 3528, 1997, 3784, 7047, 10250, 19879, 6591, 1010, 1998, 2116, 1997, 2651, 1005, 1055, 6202, 7248, 8945, 14083, 15873, 7047, 4106, 5906, 1010, 2164, 20390, 1998, 20861, 21030, 3215, 2008, 4685, 1996, 16268, 1998, 6434, 1996, 7047, 20874, 5300, 1012, 1996, 2304, 8040, 19990, 2655, 5724, 5675, 2003, 10174, 2011, 4800, 22086, 2075, 1996, 4518, 3976, 2011, 1996, 23260, 3115, 3671, 9723, 4353, 3853, 1012, 6920, 1010, 1996, 5658, 2556, 3643, 1006, 27937, 2615, 1007, 1997, 1996, 4894, 3976, 28608, 2011, 1996, 23260, 3115, 3671, 4353, 2003, 4942, 24301, 2013, 1996, 4525, 3643, 1997, 1996, 3025, 17208, 1012, 1999, 8045

In [126]:
t2 = "what is that ?"

In [127]:
doc = nlp(t2)

In [128]:
[tokens for tokens in doc if not tokens.is_punct]

[what, is, that]

In [241]:
a= [('a', 10), ('b', 11)]
b1= [('c', 11), ('a', 12)]

In [239]:
for item in a:
    if item[0] not in b1[0]:
        b1.append(item)

In [240]:
b1

[('c', 11), ('a', 12), ('a', 10), ('b', 11)]

In [11]:
x = np.array([23467.76576,76576,986876,63377,2,5,876542])

In [12]:
x

array([2.34677658e+04, 7.65760000e+04, 9.86876000e+05, 6.33770000e+04,
       2.00000000e+00, 5.00000000e+00, 8.76542000e+05])

In [16]:
ind = np.sort(x)

In [39]:
type(np.where(ind == ind[-2])[0][0])

numpy.int64